<a href="https://colab.research.google.com/github/van-dang/FEniCS-Colab/blob/master/Boussinesq_equation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import files

import platform, sys
python_version=platform.python_version()
from distutils.version import LooseVersion, StrictVersion

if ( LooseVersion(python_version) < LooseVersion("3.0.0")):
    print("Python3 is needed!");
    print("How to fix: Runtime/Change_runtime_type/Python 3");
    sys.exit()
    
try:
    from dolfin import *; from mshr import *
except ImportError as e:
    !apt-get install -y -qq software-properties-common python-software-properties module-init-tools
    !add-apt-repository -y ppa:fenics-packages/fenics
    !apt-get update -qq
    !apt install -y --no-install-recommends fenics
    from dolfin import *; from mshr import *
    
import matplotlib.pyplot as plt;
from IPython.display import clear_output, display; import time; import dolfin.common.plotting as fenicsplot 
import time

import os, sys, shutil

dolfin_version = dolfin.__version__
print ('dolfin version:', dolfin_version)

!rm -rf * # clean up all files
# Useful commands
# Remove an empty folder      : os.rmdir("my_results")
# Remove a folder with files  : shutil.rmtree("results")
# Make a folder               : os.mkdir("my_results")
# Runtime/Change_runtime_type/Python3


dolfin version: 2018.1.0


In [13]:
from dolfin import *
import mshr

# Define domain
center = Point(0.2, 0.2)
radius = 0.05
L = 2.2
W = 0.41
geometry = mshr.Rectangle(Point(0.0, 0.0), Point(L, W)) \
         - mshr.Circle(center, radius, 10)

# Build mesh
mesh = mshr.generate_mesh(geometry, 50)

# Construct facet markers
# bndry = FacetFunction("size_t", mesh)
bndry = MeshFunction("size_t", mesh, mesh.topology().dim()-1)


for f in facets(mesh):
    mp = f.midpoint()
    if near(mp[0], 0.0): # inflow
        bndry[f] = 1
    elif near(mp[0], L): # outflow
        bndry[f] = 2
    elif near(mp[1], 0.0) or near(mp[1], W): # walls
        bndry[f] = 3
    elif mp.distance(center) <= radius: # cylinder
        bndry[f] = 5

# Build function spaces (Taylor-Hood)
#V = VectorFunctionSpace(mesh, "CG", 2)
#P = FunctionSpace(mesh, "CG", 1)
#E = FunctionSpace(mesh, "CG", 1)
V = VectorElement("CG", mesh.ufl_cell(), 2)
P = FiniteElement("CG", mesh.ufl_cell(), 1)
E = FiniteElement("CG", mesh.ufl_cell(), 1)

# W = MixedFunctionSpace([V, P, E])
TH = MixedElement([V, P, E])
W = FunctionSpace(mesh, TH)

# No-slip boundary condition for velocity on walls and cylinder - boundary id 3
noslip = Constant((0, 0))
bcv_walls = DirichletBC(W.sub(0), noslip, bndry, 3)

vc= Expression(("-0.5*t*cos(atan2(x[0]-0.2,x[1]-0.2))","0.5*t*sin(atan2(x[0]-0.2,x[1]-0.2))"),t=0,degree=2)
bcv_cylinder = DirichletBC(W.sub(0), vc, bndry, 5)

bce_cylinder = DirichletBC(W.sub(2), Constant(1.0), bndry, 5)
# Inflow boundary condition for velocity - boundary id 1
v_in = Expression(("1.5 * 4.0 * x[1] * (0.41 - x[1]) / ( 0.41 * 0.41 )", "0.0"), degree=2)
bcv_in = DirichletBC(W.sub(0), v_in, bndry, 1)

# Collect boundary conditions
bcs = [bcv_cylinder, bcv_walls, bcv_in, bce_cylinder]

# Facet normal, identity tensor and boundary measure
n = FacetNormal(mesh)
I = Identity(mesh.geometry().dim())
ds = Measure("ds", subdomain_data=bndry)
nu = Constant(0.001)

dt = 0.1
t_end = 10
theta=0.5   # Crank-Nicholson timestepping
k=0.01

# Define unknown and test function(s)
(v_, p_, e_) = TestFunctions(W)

# current unknown time step
w = Function(W)
(v, p, e) = split(w)

# previous known time step
w0 = Function(W)
(v0, p0, e0) = split(w0)

def a(v,u) :
    D = sym(grad(v))
    return (inner(grad(v)*v, u) + inner(2*nu*D, grad(u)))*dx

def b(q,v) :
    return inner(div(v),q)*dx

def c(v,e,g) :
    return ( inner(k*grad(e),grad(g)) + inner(v,grad(e))*g )*dx

# Define variational forms without time derivative in previous time
F0_eq1 = a(v0,v_) + b(p,v_)
F0_eq2 = b(p_,v)
F0_eq3 = c(v0,e0,e_)
F0 = F0_eq1 + F0_eq2 + F0_eq3

# variational form without time derivative in current time
F1_eq1 = a(v,v_) + b(p,v_)
F1_eq2 = b(p_,v)
F1_eq3 = c(v,e,e_)
F1 = F1_eq1 + F1_eq2 + F1_eq3

#combine variational forms with time derivative
#
#  dw/dt + F(t) = 0 is approximated as
#  (w-w0)/dt + (1-theta)*F(t0) + theta*F(t) = 0
#
F = (inner((v-v0),v_)/dt + inner((e-e0),e_)/dt)*dx + (1.0-theta)*F0 + theta*F1

# Create files for storing solution
name="a"
#vfile = XDMFFile(mpi_comm_world(),"results_%s/v.xdmf" % name)
#pfile = XDMFFile(mpi_comm_world(),"results_%s/p.xdmf" % name)
#efile = XDMFFile(mpi_comm_world(),"results_%s/e.xdmf" % name)
#vfile.parameters["flush_output"] = True                                                               
#pfile.parameters["flush_output"] = True                                                               
#efile.parameters["flush_output"] = True                                                               

vfile = File("v.pvd");
pfile = File("p.pvd");
efile = File("e.pvd");



J = derivative(F, w)
problem=NonlinearVariationalProblem(F,w,bcs,J)
solver=NonlinearVariationalSolver(problem)


# Time-stepping
t = dt
while t < t_end:

    print("t =", t)
    vc.t=t

    # Compute
    # begin("Solving ....")
    solver.solve()
    # end()

    # Extract solutions:
    (v, p, e) = w.split()

    v.rename("v", "velocity")
    p.rename("p", "pressure")
    e.rename("e", "temperature")
    # Save to file
    vfile << v
    pfile << p
    efile << e

    # Move to next time step
    w0.assign(w)
    t += dt

t = 0.1
t = 0.2
t = 0.30000000000000004
t = 0.4
t = 0.5
t = 0.6
t = 0.7
t = 0.7999999999999999
t = 0.8999999999999999
t = 0.9999999999999999
t = 1.0999999999999999
t = 1.2
t = 1.3
t = 1.4000000000000001
t = 1.5000000000000002
t = 1.6000000000000003
t = 1.7000000000000004
t = 1.8000000000000005
t = 1.9000000000000006
t = 2.0000000000000004
t = 2.1000000000000005
t = 2.2000000000000006
t = 2.3000000000000007
t = 2.400000000000001
t = 2.500000000000001
t = 2.600000000000001
t = 2.700000000000001
t = 2.800000000000001
t = 2.9000000000000012
t = 3.0000000000000013
t = 3.1000000000000014
t = 3.2000000000000015
t = 3.3000000000000016
t = 3.4000000000000017
t = 3.5000000000000018
t = 3.600000000000002
t = 3.700000000000002
t = 3.800000000000002
t = 3.900000000000002
t = 4.000000000000002
t = 4.100000000000001
t = 4.200000000000001
t = 4.300000000000001
t = 4.4
t = 4.5
t = 4.6
t = 4.699999999999999
t = 4.799999999999999
t = 4.899999999999999
t = 4.999999999999998
t = 5.099999999999998
t = 5.199